further:
    0.0 add [goods price]
    1. one-hot encoding for reliable cate features?
    1.5. () divide [days-employed, DAYS_BIRTH]
    2. prev ratios
    3. time-related scores, and time frame
    

In [1]:
import os
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
PATH = '/home/kai/data/kaggle/homecredit/'

In [2]:
train=pd.read_pickle(PATH + 'inter/train_base.pkl')
test=pd.read_pickle(PATH + 'inter/test_base.pkl')
df = pd.concat([train,test], axis = 0)
print(df.shape)

/home/kai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  This is separate from the ipykernel package so we can avoid doing imports until


(356255, 2374)


In [12]:
def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / df[col2]

def multiply_name(col1, col2): return col1 + '_times_' + col2
def multiply(df, col1, col2): return df[col1] * df[col2]

In [3]:
train_amt = ['AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE']
divide_col = ['AMT_CREDIT_divide_AMT_ANNUITY','AMT_CREDIT_divide_AMT_GOODS_PRICE','AMT_ANNUITY_divide_DAYS_EMPLOYED','AMT_ANNUITY_divide_AMT_GOODS_PRICE']
cat_col = ['DAYS_BIRTH','DAYS_EMPLOYED']
train_col = train_amt + divide_col + cat_col

In [13]:
# install
install_amt = [col for col in df.columns if 'AMT' in col and 'install' in col and 'divide' not in col]
install_day = [col for col in df. columns if 'install' in col and 'DAY' in col and col not in install_amt]
install_sup = install_amt + install_day

for i in install_amt:
    for j in train_col:
        df[ratio_name(i,j)] = ratio(df,i,j)
print(df.shape)
for i in install_day:
    for j in train_amt:
        df[ratio_name(i,j)] = ratio(df,i,j)
print(df.shape)
for i in install_sup:
    for j in cat_col:
        df[ratio_name(i,j)] = ratio(df,i,j)
print(df.shape)

(356255, 2545)
(356255, 2662)
(356255, 2740)


In [14]:
# bureau
bureau_amt_col1 = [col for col in df.columns if 'AMT' in col and 'bureau' in col and 'mean' in col and 'PROLONG' not in col]
bureau_amt_col2 = [col for col in df.columns if 'AMT' in col and 'bureau' in col and 'min' not in col and 'PROLONG' not in col and 'divide' not in col and 'sum' not in col and 'std' not in col]
bureau_sup = bureau_amt_col1 +bureau_amt_col2
for i in bureau_sup:
    for j in train_col:
        df[ratio_name(i,j)] = ratio(df,i,j)

In [5]:
# prev
train_ori = pd.read_csv(PATH + 'application_train.csv')
prev_ori = pd.read_csv(PATH + 'previous_application.csv')
same_cols = [col for col in train_ori.columns if col in prev_ori]

In [6]:
same_cols

['SK_ID_CURR',
 'NAME_CONTRACT_TYPE',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_TYPE_SUITE',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START']

In [9]:
prev_cols = []

for i in same_cols:
    prev_cols.extend([j for j in df.columns if i in j and 'mean' in j and 'prev' in j])

In [15]:
for i in prev_cols:
    for j in train_col:
        df[ratio_name(i,j)] = ratio(df,i,j)
print(df.shape)

(356255, 3946)


In [ ]:
# [‎7/‎2/‎2018 3:12 PM]  Haoyan Cui:  
# def feature_in_time_window(df, gp_col, col, func, add_fun=None, n=None, time_col=None):
#     tmp = None
#     _df = df.copy()
#     if n is not None:
#         _df = _df[_df[time_col] >= n]
#     if add_fun is None:
#         tmp = func(_df[_df[time_col] >= n], gp_col, col)
#     else:
#         tmp = func(_df[_df[time_col] >= n], gp_col, col, add_fun)
#     columns = [str(abs(n))+'_'+x for x in set(tmp.columns) - set(gp_col)]
#     tmp.columns = columns
#     return tmp 
# func can be count, numerical, one_hot
 


In [16]:
train = df[df['TARGET'].notnull()]
test = df[df['TARGET'].isnull()]
test.drop('TARGET', inplace = True, axis = 1)
print(train.shape, test.shape)

(307511, 3946) (48744, 3945)


/home/kai/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
PATH = '/home/kai/data/kaggle/homecredit/'
print('begin cv')
target = train['TARGET']
test_df = test.copy()
ignore_cols = ['ORGANIZATION_TYPE', 'TARGET', 'SK_ID_CURR']
features = [x for x in train.columns if x not in ignore_cols]
train = train[features]
test = test[features]

from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import lightgbm as lgbm

lgbm_train = lgbm.Dataset(data=train,
                          label=target,

                          categorical_feature=[],#categorical_feats,
                          free_raw_data=False)

# lgbm_params = {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 
#           'learning_rate': 0.01, 'num_leaves': 48, 'num_iteration': 4000, 'verbose': 0 ,
#           'colsample_bytree':.8, 'subsample':.9, 'max_depth':7, 'reg_alpha':.1, 'reg_lambda':.1, 
#           'min_split_gain':.01, 'min_child_weight':1,'num_threads': 12,'feature_fraction': 0.2,
#               'scale_pos_weight':1}
lgbm_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'num_iteration': 4000,
    'num_threads': 8,

    'num_leaves': int(round(44.368535336628419)),
    'feature_fraction': 0.28231763168020257,
    'bagging_fraction': 0.94901525271474951,
    'max_depth': int(round(8.0430115561596267)),
    'lambda_l1': 0.30680079516647751,
    'lambda_l2': 0.079128660903201031,
    'min_split_gain': 0.054005067457890979,
    'min_child_weight': 98.172643147364937}


cv_results = lgbm.cv(train_set=lgbm_train,
                     params=lgbm_params,
                     nfold=5,
                     early_stopping_rounds=150,
                     verbose_eval=100,
                     metrics=['auc'])

optimum_boost_rounds = np.argmax(cv_results['auc-mean'])
print('Optimum boost rounds = {}'.format(optimum_boost_rounds))
print('Best CV result = {}'.format(np.max(cv_results['auc-mean'])))

clf = lgbm.train(train_set=lgbm_train,
                 params=lgbm_params,
                 num_boost_round=optimum_boost_rounds,
                verbose_eval=100)

""" Predict on test set and create submission """
y_pred = clf.predict(test)
out_df = pd.DataFrame({'SK_ID_CURR': test_df['SK_ID_CURR'], 'TARGET': y_pred})
#     small_len = out_df[out_df['TARGET']< threshold1]
#     large_len = out_df[out_df['TARGET']> threshold2]
#     print('length of small and large,',len(small_len),len(large_len))
print((out_df['TARGET']>0.5).sum(),'more than half')

begin cv


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:390: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[100]	cv_agg's auc: 0.780524 + 0.00302028
[200]	cv_agg's auc: 0.789798 + 0.00267691
[300]	cv_agg's auc: 0.79263 + 0.00240452
[400]	cv_agg's auc: 0.793662 + 0.00238787
[500]	cv_agg's auc: 0.79388 + 0.0023581
[600]	cv_agg's auc: 0.793698 + 0.00228649
Optimum boost rounds = 483
Best CV result = 0.7939317310483928


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
